In [ ]:
# default_exp model.core

# Model

> API details. @nathan

In [10]:
# export
import json
import logging

import tensorflow as tf

from abc import ABC, abstractmethod
from pathlib import Path
from tokenizers import Tokenizer

import datetime

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [11]:
from transformers import TFGPT2LMHeadModel, GPT2Config, GPT2Tokenizer

In [12]:
import time

In [4]:
# hide
# Setup
import numpy as np
import pandas as pd

from icodegen.data.core import convert_df_to_tfds, java_special_tokens, train_tokenizer

df_fake = pd.DataFrame(
    ["aaaa(bb(aaaa(bb()()ccc)dd)()ccc)dd", "aaaa(bb()ccccc)dd"], columns=["code"]
)

# Tokenize the data
max_length = 16
batch_size = 1
vocab_sz = 100
tokenizer = train_tokenizer(df_fake, java_special_tokens, max_length, vocab_sz=vocab_sz)
dataset = convert_df_to_tfds(df_fake, tokenizer, max_length, batch_size)

Load Codesearchnet Java

In [423]:
java_df = pd.read_csv('/tf/main/dvc-icodegen/data/clean_java.csv')

In [424]:
java_df.head()

,Unnamed: 0,code,code_len,code_tokens,cyclomatic_complexity,data_type,method_name,nloc,parameter_count,partition,token_count
0,0,protected final void fastPathOrderedEmit(U val...,134.0,"['protected', 'final', 'void', 'fastPathOrdere...",7.0,src,fastPathOrderedEmit,20.0,3.0,test,131.0
1,1,@CheckReturnValue\n @NonNull\n @Schedule...,63.0,"['@', 'CheckReturnValue', '@', 'NonNull', '@',...",1.0,src,amb,4.0,1.0,test,43.0
2,2,"@SuppressWarnings(""unchecked"")\n @CheckRetu...",107.0,"['@', 'SuppressWarnings', '(', '""unchecked""', ...",3.0,src,ambArray,11.0,1.0,test,82.0
3,3,"@SuppressWarnings({ ""unchecked"", ""rawtypes"" })...",79.0,"['@', 'SuppressWarnings', '(', '{', '""unchecke...",1.0,src,concat,4.0,1.0,test,50.0
4,4,"@SuppressWarnings({ ""unchecked"", ""rawtypes"" })...",91.0,"['@', 'SuppressWarnings', '(', '{', '""unchecke...",1.0,src,concat,5.0,2.0,test,62.0


In [425]:
java_df.describe()

,Unnamed: 0,code_len,cyclomatic_complexity,nloc,parameter_count,token_count
count,496494.000000,496494.00000,496494.000000,496494.000000,496494.000000,496494.000000
mean,14574.923598,112.20115,3.784144,14.915854,1.633015,106.590708
std,8655.530474,227.68705,12.617511,31.642129,1.390480,221.178693
min,0.000000,20.00000,1.000000,2.000000,0.000000,7.000000
25%,7041.000000,42.00000,1.000000,6.000000,1.000000,38.000000
50%,14346.000000,66.00000,2.000000,9.000000,1.000000,62.000000
75%,22046.000000,121.00000,4.000000,16.000000,2.000000,116.000000
max,29999.000000,68278.00000,4434.000000,11325.000000,54.000000,68274.000000


In [427]:
java_df.shape

(496494, 11)

In [426]:
java_train = java_df[java_df['partition'] == 'train']

In [428]:
java_train.shape

(454273, 11)

In [429]:
java_trn_samples = java_train.sample(100000)

In [430]:
# Tokenize the data

max_length = 112
batch_size = 16
vocab_sz = 10000
tokenizer = train_tokenizer(java_trn_samples, java_special_tokens, max_length, vocab_sz=vocab_sz)

In [432]:
java_trn_dataset = convert_df_to_tfds(java_trn_samples, tokenizer, max_length, batch_size)

In [433]:
java_trn_dataset

<BatchDataset shapes: ((16, 111), (16, 111)), types: (tf.int32, tf.int32)>

In [431]:
tokenizer.get_vocab_size()

10000

In [5]:
# export
def _loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True
    )

In [13]:
# export
class Model(ABC):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    @abstractmethod
    def from_path(path):
        pass

    @abstractmethod
    def get_probs(self, inputs):
        pass

    @abstractmethod
    def save(self, path):
        pass

    @abstractmethod
    def tokenize(self, method):
        pass

    @abstractmethod
    def train(self, ds, epochs):
        pass

# Transformer Models

HuggingFace-based implementation, originally [this](!https://huggingface.co/transformers/model_doc/gpt2.html).

In [240]:
# export

class TransformerHFModel(Model):
    def __init__(
        self,
        out_path,
        model,
        tokenizer,
        optimizer,
        loss
    ):
        """
        Constructs a custom transformer model using a provided HF model
        :param out_path: 
        :param model: HF transformer model
        :tokenizer: HF tokenizer model
        :optimizer: Keras optimizer
        :loss: Custom loss function for model training
        """
        
        # HuggingFace model implementation is left outside this class in order to
        # decouple from any specific implementation, besides aiming experimentation with multiple models
        
        self.timestamp = f'{model.name}-{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
        self.out_path = Path(out_path) / self.timestamp
        self.out_path.mkdir(exist_ok=True)
        self.optimizer = optimizer
        self.loss = loss
        
        tensorboard_path = self.out_path / "tensorboard_logs"
        tensorboard_path.mkdir(exist_ok=True)
        tb_logs_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_path = tensorboard_path  / tb_logs_timestamp
        
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=str(tensorboard_path),
                histogram_freq=0,  # How often to log histogram visualizations
                embeddings_freq=0,  # How often to log embedding visualizations
                update_freq="epoch",
            ),  # How often to write logs (default: once per epoch)
            tf.keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=1e-2,
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
            ),
        ]
        
        super().__init__(tokenizer, model)
        
    @staticmethod
    def from_path(model_path, hf_model, optimizer, loss):
        path = Path(model_path)
        
        # Load tokenizer
        tokenizer = Tokenizer.from_file(str(path / "tokenizer.json"))
        
        # Load model
        with open(path / "model_config.json", "r") as f:
            model_config = json.load(f)
        full_path = path / "model"
        loaded_model = hf_model.from_pretrained(full_path)
        
        model = TransformerHFModel(model_path, loaded_model, tokenizer, optimizer, loss)
        
        return model
    
    def save(self):
        # Save tokenizer
        self.tokenizer.save(str(self.out_path / "tokenizer.json"), pretty=True)
        # Save HF model
        self.model.save_pretrained(self.out_path / "model")
        
        # Save model config.
        model_config = self.model.config
        # TODO: extract attributes from configuration
        config_dict = {
          "activation_function": model_config.activation_function,
          "attn_pdrop": model_config.attn_pdrop,
          "bos_token_id": model_config.bos_token_id,
          "embd_pdrop": model_config.embd_pdrop,
          "eos_token_id": model_config.eos_token_id,
          "gradient_checkpointing": model_config.gradient_checkpointing,
          "initializer_range": model_config.initializer_range,
          "layer_norm_epsilon": model_config.layer_norm_epsilon,
          "model_type": model_config.model_type,
          "n_ctx": model_config.n_ctx,
          "n_embd": model_config.n_embd,
          "n_head": model_config.n_head,
          "n_inner": model_config.n_inner,
          "n_layer": model_config.n_layer,
          "n_positions": model_config.n_positions,
          "pad_token_id": model_config.pad_token_id,
          "resid_pdrop": model_config.resid_pdrop,
          "summary_activation": model_config.summary_activation,
          "summary_first_dropout": model_config.summary_first_dropout,
          "summary_proj_to_labels": model_config.summary_proj_to_labels,
          "summary_type": model_config.summary_type,
          "summary_use_proj": model_config.summary_use_proj,
          "vocab_size": model_config.vocab_size
        }
        
        with open(self.out_path / "model_config.json", "w") as file:
            json.dump(config_dict, file)
        
    
    def generate(self, max_length, n, top_k=10, temperature=1.0):
        """
        Performs generation process through sampling strategy (top-k) by default
        :param max_length: Max. allowed length for generated sequences
        :param n: Number of samples to generate
        :param top_k: top-k most likely next words to be filtered for the sampling
        :returns Generated sequences:
        """
        
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)
        
        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_k=top_k,
            num_return_sequences=n
        )
        
        generated = []
        for i, sample in enumerate(gen_txt):
            generated.append(self.tokenizer.decode(sample, skip_special_tokens=False))
            
        return generated
    
    def get_probs(self, inputs):
        outputs = self.model(inputs)
        logits = outputs[0]
        probs = tf.nn.softmax(logits)

        return probs    
    
    def sample_top_k(self, max_length, top_k=50):
        """
        Performs sampling (generation) using top-k mechanism
        :param max_length: Max. length allowed in the generation process
        :param top_k: top-k most likely next words to be filtered for the sampling
        :returns: Generated text
        """
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)
        
        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_k=top_k
        )
        
        return self.tokenizer.decode(gen_txt[0], skip_special_tokens=False)
    
    def sample_nucleus(self, max_length, top_p=0.9):
        """
        Perform sampling using top-p (nucleus) mechanism
        :param  max_length: Max. allowed length of generated sequences 
        :param top_p: Probability value to find possible set of words whose cumulative probability exceeds top-p value
        :returns: Generated text
        """
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)

        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_p=top_p,
            top_k=0
        )

        return self.tokenizer.decode(gen_txt[0], skip_special_tokens=False)
    
    def sample_temperature(self, max_length, temperature=0.7):
        """
        Performs sampling using temperature strategy
        :param max_length: Max. allowed length of generated sequences 
        :param temperature: Temperature parameter of the softmax
        :returns: Generated text
        """
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)

        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_k=0,
            temperature=temperature
        )
        
        return self.tokenizer.decode(gen_txt[0], skip_special_tokens=False)
        

    def tokenize(self, method):
        """
        :param method: Code snippet (plain text)
        :returns: Encoded result using the provided tokenizer
        """
        output = {}
        # encode method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )

        return output

        # return self.tokenizer(method, return_tensors="tf")

    def train(self, ds_train, ds_val, epochs):
        """
        Performs training process leveraging keras methods.
        :param ds_train: Tensorflow dataset for training
        :param ds_val: Tensorflow dataset for validation
        :param epochs: Number of epochs to execute training
        """
        # Be careful with the definition of the loss function when compiling
        # Gotta consider the number of layers (n_layer param.) --> actually number of decoder blocks
        self.model.compile(optimizer=self.optimizer, loss=[self.loss, *[None] * self.model.config.n_layer])
        history = self.model.fit(ds_train, epochs=epochs, callbacks=self.callbacks, validation_data=ds_val)
        
        return history

Test GPT2 - HuggingFace-based

In [434]:
# Establish configuration for HF model, in this case by means of GPT2Config class

custom_config_gpt2 = GPT2Config(
    vocab_size = tokenizer.get_vocab_size(),
    bos_token_id = tokenizer.token_to_id("<sos>"),
    eos_token_id = tokenizer.token_to_id("<eos>"),
    pad_token_id = tokenizer.token_to_id("<pad>")
)

In [435]:
# Configuration

gpt2_hf_model_path = "models/GPT2-HF"

In [436]:
# Implement the corresponding HuggingFace model using the custom configuration

hf_gpt_model = TFGPT2LMHeadModel(custom_config_gpt2)

In [437]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

In [438]:
gpt2_model = TransformerHFModel(gpt2_hf_model_path, hf_gpt_model, tokenizer, optimizer, _loss)

In [447]:
TRAIN_EPOCHS = 20

In [448]:
start_trn_time = time.time()
gpt2_model.train(dataset, dataset, epochs=TRAIN_EPOCHS)
end_trn_time = time.time()
print(f'Time elapsed training: {end_trn_time - start_trn_time} seconds.')

Epoch 1/20
6250/6250 [==============================] - 1210s 192ms/step - loss: 3.6715 - output_1_loss: 3.6715 - val_loss: 2.4627 - val_output_1_loss: 2.4627
Epoch 2/20
6250/6250 [==============================] - 1199s 192ms/step - loss: 2.4219 - output_1_loss: 2.4219 - val_loss: 1.9568 - val_output_1_loss: 1.9568
Epoch 3/20
6250/6250 [==============================] - 1199s 192ms/step - loss: 1.9869 - output_1_loss: 1.9869 - val_loss: 1.6407 - val_output_1_loss: 1.6407
Epoch 4/20
6250/6250 [==============================] - 1199s 192ms/step - loss: 1.7135 - output_1_loss: 1.7135 - val_loss: 1.4658 - val_output_1_loss: 1.4658
Epoch 5/20
6250/6250 [==============================] - 1199s 192ms/step - loss: 1.5522 - output_1_loss: 1.5522 - val_loss: 1.3510 - val_output_1_loss: 1.3510
Epoch 6/20
6250/6250 [==============================] - 1199s 192ms/step - loss: 1.4429 - output_1_loss: 1.4429 - val_loss: 1.2603 - val_output_1_loss: 1.2603
Epoch 7/20
6250/6250 [========================

In [451]:
NUMBER_GEN_SEQS = 10

In [452]:
start_gen_time = time.time()
generated_seqs = gpt2_model.generate(n=NUMBER_GEN_SEQS, max_length=max_length)
end_gen_time = time.time()
print(f'Time elapsed generating: {end_gen_time - start_gen_time} seconds.')

Time elapsed generating: 10.738839387893677 seconds.


In [454]:
for seq in generated_seqs:
    print(f'sample: {seq}')

sample: <sos>public void marshall(DomainName domainName, ProtocolMarshaller protocolMarshaller) {

        if (domainName == null) {
            throw new SdkClientException("Invalid argument passed to marshall(...)");
        }

        try {
            protocolMarshaller.marshall(domainName.getDomainName(), DOMAINNAME_BINDING);
            protocolMarshaller.marshall(domainName.getWebDomainName(), APPROMAINNAME_BINDING);
        } catch (Exception e) {
            throw new SdkClientException("Unable to marshall request to JSON: " + e.
sample: <sos>public void setAttribute(String key, Object attribute)
    {
        if (this.element != null)
            this.element.attributes.put(key, attribute);
    }<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [250]:
gpt2_model.save()

In [251]:
loaded_gpt2_model = TransformerHFModel.from_path("models/GPT2-HF/tfgp_t2lm_head_model_4-20210330-171318", TFGPT2LMHeadModel, optimizer, _loss)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at models/GPT2-HF/tfgp_t2lm_head_model_4-20210330-171318/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [252]:
type(loaded_gpt2_model)

__main__.TransformerHFModel

In [253]:
generated_seqs = gpt2_model.generate(n=10, max_length=max_length)
print(generated_seqs)

['<sos>aabbaa(aaaa(b)<assert>()baa', '<sos>aa(aa(()aabdbbcbc)', '<sos>aaaab<assert>c(bbbcccc))', '<sos>aaaaaa(bbc(cc(()((', '<sos>aaaa(aaccaa(c)caac))', '<sos>aa(aa(b)bb(aac)c)<static>', '<sos>aabcbbaabaa(((()))', '<sos>aa(b(aaaa(b(bbc(b(', '<sos>aaaac(aabbc(d(aa))(', '<sos>aaaa()(cbbbc(aac(b']


In [254]:
text = "test"
encoded_text = tokenizer.encode("<sos>" + text).ids
input_eval = tf.expand_dims(encoded_text, 0)
logits = gpt2_model.model(input_eval)[0].numpy()
inputs = gpt2_model.tokenize(text)
probs = gpt2_model.get_probs(inputs)[0].numpy()

In [255]:
probs.shape

(16, 100)

In [49]:
# for i in range(len(probs)):
#     assert np.isclose(1.0, probs[i].sum())
#     assert np.argmax(logits[i]) == np.argmax(probs[i])

In [256]:
gpt2_model.sample_nucleus(max_length=20)

'<sos>aaaaaa(</>aa))b(((<||>ccaacb('

In [257]:
gpt2_model.sample_top_k(max_length=20, top_k=10)

'<sos>aabaaaa((b()ccb))((c(('

Keras-based implementation (based on [this example](!https://keras.io/examples/generative/text_generation_with_miniature_gpt/#prepare-the-data-for-wordlevel-language-modelling))

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

Auxiliar classes / methods for building the model.

In [6]:
# export

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

In [7]:
# export

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        return {
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        }
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
    
# export    class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        return {
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        }
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [8]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        return {
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim
        }
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [9]:
# export

class MiniatureGPTModel(Model):
    def __init__(
        self,
        out_path, 
        max_length,
        batch_size,
        ff_dim,
        embedding_dim,
        n_heads,
        n_transformer_blocks,
        vocab_size,
        tokenizer,
        optimizer,
        loss
    ):
        """
        Builds a transformer model using tf-keras API
        :param out_path: Path for saving model data
        :param max_length: Max. length allowed for the sequences
        :param batch_size: 
        :param ff_dim:
        :param embedding_dim:
        :param n_heads:
        :param n_transformer_blocks:
        :param vocab_size:
        :param tokenizer:
        :param optimizer:
        :param loss:
        """
        self.max_length = max_length
        self.batch_size = batch_size
        self.ff_dim = ff_dim
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_transformer_blocks = n_transformer_blocks
        self.vocab_size = vocab_size
        self.loss = loss
        self.optimizer = optimizer
        
        model = self.__create_model()
        
        model._name = "Miniature-GPT"
        
        name = f'{model._name}-{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
        self.out_path = Path(out_path)/name
        self.out_path.mkdir(exist_ok=True)
        
        tensorboard_path = self.out_path / "tensorboard_logs"
        tensorboard_path.mkdir(exist_ok=True)
        tensorboard_path = tensorboard_path / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=str(tensorboard_path),
                histogram_freq=0,  # How often to log histogram visualizations
                embeddings_freq=0,  # How often to log embedding visualizations
                update_freq="epoch",
            ),  # How often to write logs (default: once per epoch)
            tf.keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=1e-2,
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
            ),
        ]

        super().__init__(tokenizer, model)
     
    def __create_model(self):
        inputs = layers.Input(shape=(self.max_length-1,), dtype=tf.int32)
        embedding_layer = TokenAndPositionEmbedding(self.max_length-1, self.vocab_size, self.embedding_dim)
        x = embedding_layer(inputs)
        transformerLayers = []
        outputs_x = []
        for i in range(self.n_transformer_blocks):
            transformerLayers.append(TransformerBlock(self.embedding_dim, self.n_heads, self.ff_dim))

        for i in range(self.n_transformer_blocks):
            x = transformerLayers[i](x)
            outputs_x.append(x)
        # transformer_block = TransformerBlock(self.embedding_dim, self.n_heads, self.ff_dim)
        # x = transformer_block(x)

        outputs = layers.Dense(self.vocab_size)(outputs_x[self.n_transformer_blocks-1])
        # outputs = layers.Dense(self.vocab_size)(x)
        model = keras.Model(inputs=inputs, outputs=[outputs, *outputs_x])
        
        return model
        
    @staticmethod
    def from_path(path, optimizer, loss):
        path = Path(path)
        # Load tokenizer
        tokenizer = Tokenizer.from_file(str(path / "tokenizer.json"))
        # Load model
        with open(path / "model_config.json", "r") as f:
            model_config = json.load(f)
            
        min_model = MiniatureGPTModel(
            path,
            **model_config,
            tokenizer=tokenizer,
            optimizer=optimizer, 
            loss=loss
        )
        
        loaded_model = tf.keras.models.load_model(str(path), custom_objects={"_loss": _loss})
        min_model.model = loaded_model
        
        return min_model
        
    def save(self):
        # Save tokenizer
        self.tokenizer.save(str(self.out_path / "tokenizer.json"), pretty=True)
        # Save model
        self.model.save(str(self.out_path))
        # Save config.
        model_config = {
            "max_length": self.max_length,
            "batch_size": self.batch_size,
            "ff_dim": self.ff_dim,
            "embedding_dim": self.embedding_dim,
            "n_heads": self.n_heads,
            "n_transformer_blocks": self.n_transformer_blocks,
            "vocab_size": self.vocab_size,
        }
        with open(self.out_path / "model_config.json", "w") as file:
            json.dump(model_config, file)
        
    def tokenize(self, method):
        """
        :param method: Code snippet
        :returns: Encoded result using the provided tokenizer
        """
        output = {}
        # encode method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )

        return output
    
    def generate(self, n, top_k=10):
        """Performs generation process through top-k sampling process"""
         # Converting our start string to numbers (vectorizing)
        start_tokens = [self.tokenizer.encode("<sos>").ids[0]]
        print(start_tokens)
        generated_txt = []
        
        while len(generated_txt) < n:
            gen_sample = self.__sample_top_k(start_tokens, top_k)
            generated_txt.append(gen_sample)
            
        # TODO: Tokenize
        return generated_txt
        
    def __sample_top_k(self, start_tokens, top_k=10):
        num_tokens_generated = 0
        tokens_generated = [] 
        gen_sequence = start_tokens[:]
        while len(gen_sequence) <= self.max_length-1:
            pad_len = (self.max_length-1) - len(gen_sequence)
            sample_index = len(gen_sequence) - 1
            if pad_len < 0:
                x = gen_sequence[:self.max_length-1]
                sample_index = self.max_length - 1
            elif pad_len > 0:
                x = gen_sequence + [0] * pad_len
            else:
                x = gen_sequence[:]
            sample_index = len(gen_sequence)-1
            x = np.array([x])
            prediction_output = self.model.predict(x)
            y = prediction_output[0]
            sample_token = self.__sample_from(y[0][sample_index], top_k)
            tokens_generated.append(sample_token)
            gen_sequence.append(sample_token)
            num_tokens_generated = len(tokens_generated)
            
        return tokens_generated
            
    def __sample_from(self, logits, top_k):
        logits, indices = tf.math.top_k(logits, k=top_k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)
    
    def get_probs(self, inputs):
        #         ids = self.tokenizer.encode("<sos>" + method).ids
        #         input_eval = tf.expand_dims(ids, 0)

        logits = self.model(inputs["input_ids"])
        probs = tf.nn.softmax(logits)  # [0].numpy()

        return probs
        
    def train(self, ds_train, ds_val, epochs):
        """
        Performs training process leveraging keras methods
        """
        self.model.compile(optimizer=self.optimizer, loss=[self.loss, *[None]* self.n_transformer_blocks])
        history = self.model.fit(ds_train, epochs=epochs, callbacks=self.callbacks, validation_data=ds_val)
        
        return history

NameError: name 'Model' is not defined

Test Miniature GPT. Keras-based

In [455]:
# Configuration parameters
# 
min_gpt_config_params = {
    "out_path": "models/Min-GPT",
    "max_length" : max_length,
    "batch_size": batch_size,
    "ff_dim": 64,
    "embedding_dim": 128,
    "n_heads": 2,
    "n_transformer_blocks": 1,
    "vocab_size": tokenizer.get_vocab_size()
}

In [456]:
# Optimizer, loss definition

min_gpt_optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
min_gpt_loss = _loss

In [457]:
min_gpt_config_params["optimizer"] = min_gpt_optimizer
min_gpt_config_params["loss"] = min_gpt_loss
min_gpt_config_params["tokenizer"] = tokenizer

In [458]:
min_gpt_model = MiniatureGPTModel(**min_gpt_config_params)

In [459]:
start_trn_min_time = time.time()
min_gpt_history = min_gpt_model.train(dataset, dataset, epochs=TRAIN_EPOCHS)
end_trn_min_time = time.time()
print(f'Time elapsed: {end_trn_min_time - start_trn_min_time} seconds')

Epoch 1/20
6250/6250 [==============================] - 100s 16ms/step - loss: 6.0205 - dense_103_loss: 6.0205 - val_loss: 3.6836 - val_dense_103_loss: 3.6836
Epoch 2/20
6250/6250 [==============================] - 97s 16ms/step - loss: 3.5820 - dense_103_loss: 3.5820 - val_loss: 3.3297 - val_dense_103_loss: 3.3297
Epoch 3/20
6250/6250 [==============================] - 98s 16ms/step - loss: 3.2802 - dense_103_loss: 3.2802 - val_loss: 3.1048 - val_dense_103_loss: 3.1048
Epoch 4/20
6250/6250 [==============================] - 97s 16ms/step - loss: 3.0777 - dense_103_loss: 3.0777 - val_loss: 2.9419 - val_dense_103_loss: 2.9419
Epoch 5/20
6250/6250 [==============================] - 98s 16ms/step - loss: 2.9302 - dense_103_loss: 2.9302 - val_loss: 2.8194 - val_dense_103_loss: 2.8194
Epoch 6/20
6250/6250 [==============================] - 97s 16ms/step - loss: 2.8198 - dense_103_loss: 2.8198 - val_loss: 2.7255 - val_dense_103_loss: 2.7255
Epoch 7/20
6250/6250 [=============================

In [351]:
dataset

<BatchDataset shapes: ((1, 15), (1, 15)), types: (tf.int32, tf.int32)>

In [2]:
min_gpt_model.model.summary()

NameError: name 'min_gpt_model' is not defined

In [461]:
gen_sequences = min_gpt_model.generate(n=10)

[1]


In [465]:
for seq in gen_sequences:
    print('-'*100)
    print(f'seq: {tokenizer.decode(seq, skip_special_tokens=False)}')

----------------------------------------------------------------------------------------------------
seq: private int getMaxSize() {
        long totalSize = Math.minTime;
        int numRows = -1];
        for (int i = 0; i < maxNumsize; i++) {
            if (i > 1) {
                if (currentCapacity > MAX_VALUE) {
                    currentTime = currentTime;
                } else if (lastSize <= 0) {
                    m_index = new IndexOutOfBoundsException();
                    if (size() != - 1) {
                        return - 1
----------------------------------------------------------------------------------------------------
seq: @Override
	public void set_PUT_First(long value)
		throws ServletException, IOException
	{
		if (value == value.value_value = new Long();
		if (value == null) {
			return value;
		}
	}<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [409]:
min_gpt_model.save()

INFO:tensorflow:Assets written to: models/Min-GPT/Miniature-GPT-20210330-200608/assets


INFO:tensorflow:Assets written to: models/Min-GPT/Miniature-GPT-20210330-200608/assets


In [421]:
loaded_min_gpt = MiniatureGPTModel.from_path("models/Min-GPT/Miniature-GPT-20210330-200608", optimizer, min_gpt_loss)

In [422]:
type(loaded_min_gpt)

__main__.MiniatureGPTModel

# RNN models

In [9]:
# export
class RNNModel(Model):
    _RNN_TYPE = {
        "rnn": tf.keras.layers.SimpleRNN,
        "gru": tf.keras.layers.GRU,
        "lstm": tf.keras.layers.LSTM,
    }

    def __init__(
        self,
        rnn_type,
        n_layers,
        vocab_size,
        embedding_dim,
        rnn_units,
        batch_size,
        out_path,
        tokenizer,
    ):
        self.rnn_type = rnn_type
        self.n_layers = n_layers
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.rnn_units = rnn_units

        self.config_name = f"{rnn_type}_layers{n_layers}_vocab{vocab_size}_embed{embedding_dim}_units{rnn_units}"
        self.out_path = Path(out_path) / self.config_name
        self.out_path.mkdir(exist_ok=True)
        tensorboard_path = self.out_path / "tensorboard_logs"
        tensorboard_path.mkdir(exist_ok=True)
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=str(tensorboard_path),
                histogram_freq=0,  # How often to log histogram visualizations
                embeddings_freq=0,  # How often to log embedding visualizations
                update_freq="epoch",
            ),  # How often to write logs (default: once per epoch)
            tf.keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=1e-2,
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
            ),
        ]

        layer = RNNModel._RNN_TYPE[rnn_type]
        rnn_layers = [
            layer(
                rnn_units,
                return_sequences=True,
                recurrent_initializer="glorot_uniform",
                # following BigCode != Big Vocab Paper
                dropout=0.5,
            )
            for _ in range(n_layers)
        ]
        model = tf.keras.Sequential(
            [
                tf.keras.layers.Embedding(
                    input_dim=vocab_size,
                    output_dim=embedding_dim,
                    mask_zero=True,  # Zero cannot be used in the vocabulary
                ),
            ]
            + rnn_layers
            + [
                tf.keras.layers.Dense(vocab_size),
            ]
        )

        super().__init__(tokenizer, model)

    @staticmethod
    def from_path(path):
        path = Path(path)

        tokenizer = Tokenizer.from_file(str(path / "tokenizer.json"))
        with open(path / "model_config.json", "r") as f:
            model_config = json.load(f)

        model = RNNModel(
            model_config["rnn_type"],
            model_config["n_layers"],
            model_config["vocab_size"],
            model_config["embedding_dim"],
            model_config["rnn_units"],
            1,
            path,
            tokenizer,
        )
        model.model = tf.keras.models.load_model(
            str(path), custom_objects={"_loss": _loss}
        )

        return model

    def get_probs(self, inputs):
        #         ids = self.tokenizer.encode("<sos>" + method).ids
        #         input_eval = tf.expand_dims(ids, 0)

        logits = self.model(inputs["input_ids"])
        probs = tf.nn.softmax(logits)  # [0].numpy()

        return probs

    def generate(self, n, temperature=1.0):
        # Converting our start string to numbers (vectorizing)
        text_generated = [self.tokenizer.encode("<sos>").ids[0]]
        input_eval = tf.expand_dims(text_generated, 0)

        # Here batch size == 1
        self.model.reset_states()
        for i in range(n):
            predictions = self.model(input_eval)
            # remove the batch dimension
            predictions = tf.squeeze(predictions, 0)

            # using a categorical distribution to predict the character
            # returned by the model
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[
                -1, 0
            ].numpy()

            text_generated.append(predicted_id)
            # Pass the predicted character as the next input to the model
            # along with the previous hidden state
            input_eval = tf.expand_dims(text_generated, 0)

        return self.tokenizer.decode(text_generated, skip_special_tokens=False)

    def save(self):
        self.tokenizer.save(str(self.out_path / "tokenizer.json"), pretty=True)
        self.model.save(str(self.out_path))
        model_config = {
            "rnn_type": self.rnn_type,
            "n_layers": self.n_layers,
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "rnn_units": self.rnn_units,
        }
        with open(self.out_path / "model_config.json", "w") as f:
            json.dump(model_config, f)

    def tokenize(self, method):
        #         ids = self.tokenizer.encode("<sos>" + method).ids
        #         inputs = tf.expand_dims(ids, 0)
        output = {}
        # encod method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )
        return output  # self.tokenizer(method, return_tensors="tf")

    # TODO add tensorboard call back for easy visualization
    def train(self, ds_trn, ds_val, epochs):
        self.model.compile(optimizer="adam", loss=_loss)
        history = self.model.fit(
            ds_trn, epochs=epochs, callbacks=self.callbacks, validation_data=ds_val
        )

        return history

In [10]:
PARAM_COUNT = 124_772

rnn_type = "gru"
n_layers = 1
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 128
rnn_units = 128
batch_size = 1
out_path = "/tmp"
gru = RNNModel(
    rnn_type,
    n_layers,
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size,
    out_path,
    tokenizer,
)

assert PARAM_COUNT == gru.model.count_params()

In [17]:
from transformers import GPT2Config

In [18]:
gpt2_configuration = GPT2Config()

In [ ]:
TFLM

In [39]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [51]:
gpt2_model = TransformerModel('gpt2_model', , tokenizer, optimizer, _loss)

In [ ]:
transformer_history = gpt2_model.train(dataset, dataset, EPOCHS)

In [54]:
gpt2_model.generate(10, 10)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


'<sos>'

In [11]:
EPOCHS = 1
chkpt_path = Path(out_path) / (
    f"{rnn_type}_layers{n_layers}_vocab{vocab_size}_embed{embedding_dim}_units{rnn_units}"
)
history = gru.train(dataset, dataset, EPOCHS)

assert chkpt_path.exists()
assert EPOCHS == len(list(chkpt_path.glob("*.index")))
assert EPOCHS == len(history.history["loss"])

2/2 [==============================] - 8s 2s/step - loss: 4.6063 - val_loss: 4.5652


In [ ]:
# How do I test this?
text = "test"
text_generated = tokenizer.encode("<sos>" + text).ids
input_eval = tf.expand_dims(text_generated, 0)

logits = gru.model(input_eval)[0].numpy()
inputs = gru.tokenize(text)
probs = gru.get_probs(inputs)[0].numpy()

for i in range(len(probs)):
    assert np.isclose(1.0, probs[i].sum())
    assert np.argmax(logits[i]) == np.argmax(probs[i])

In [ ]:
# Add test case for earlystopping

In [ ]:
# # this test will break sometimes due to encoding adding space prefixes to some
# # tokens when add_prefix_space=False it should be fine :/
# NUM_TOKENS = 10
# text = gru.generate(NUM_TOKENS)
# tokenizer.no_padding()
# ids = tokenizer.encode(text).ids
# tokenizer.enable_padding(length=max_length)

# # -1 for the <sos> token that's always prepended
# assert NUM_TOKENS == len(ids) - 1

In [ ]:
gru.save()

assert (gru.out_path / "assets").exists()
assert (gru.out_path / "model_config.json").exists()
assert (gru.out_path / "saved_model.pb").exists()
assert (gru.out_path / "tokenizer.json").exists()
assert (gru.out_path / "variables").exists()

INFO:tensorflow:Assets written to: /tmp/gru_layers1_vocab100_embed128_units128/assets


INFO:tensorflow:Assets written to: /tmp/gru_layers1_vocab100_embed128_units128/assets


In [ ]:
loaded_gru = RNNModel.from_path(str(gru.out_path))

assert gru.tokenizer.get_vocab() == loaded_gru.tokenizer.get_vocab()
assert gru.model.count_params() == loaded_gru.model.count_params()
assert gru.model.evaluate(dataset, verbose=2) == loaded_gru.model.evaluate(
    dataset, verbose=2
)

2/2 - 0s - loss: 4.5705
2/2 - 1s - loss: 4.5705


In [ ]:
# export

# Experiment 0.0.0
VANILLA_CONFIG = {
    "rnn_type": "rnn",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.0.0
GRU_CONFIG_1 = {
    "rnn_type": "gru",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.1.0
GRU_CONFIG_2 = {
    "rnn_type": "gru",
    "n_layers": 2,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.1.1
GRU_CONFIG_3 = {
    "rnn_type": "gru",
    "n_layers": 3,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.2.0
GRU_CONFIG_4 = {
    "rnn_type": "gru",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 512,
}

# Experiment 1.2.1
GRU_CONFIG_5 = {
    "rnn_type": "gru",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 2_048,
}

_RNN_CONFIGs = [
    VANILLA_CONFIG,
    GRU_CONFIG_1,
    GRU_CONFIG_2,
    GRU_CONFIG_3,
    GRU_CONFIG_4,
    GRU_CONFIG_5,
]

In [ ]:
# export
def train(data_path, out_path, epochs=64, max_length=300, batch_size=64, n=None):
    """Function for training models related to the library."""
    out_path.mkdir(exist_ok=True)

    # Load in the datasets

    # Train BPE tokenizer
    # Check if the path where the tokenizer is to be saved is not empty
    # if it is not empty then just load the tokenizer there.
    if (out_path / "tokenizer.json").exists():
        logging.info(f"Loading tokenizer from {str(out_path / 'tokenizer.json')}.")
        tokenizer = Tokenizer.from_file(str(out_path / "tokenizer.json"))
    else:
        logging.info(
            f"Training new tokenizer and saving to {str(out_path / 'tokenizer.json')}."
        )
        df_bpe = pd.read_json(
            data_path / "codesearchnet_java" / "bpe.jsonl", orient="records", lines=True
        )[:n]
        tokenizer = train_tokenizer(df_bpe, java_special_tokens, max_length)
        tokenizer.save(str(out_path / "tokenizer.json"), pretty=True)
        del df_bpe

    # Tokenize the dataset and convert it to tfds.
    tfds_trn_path = (
        data_path / "codesearchnet_java" / f"tfds_trn_{max_length}len_{batch_size}bs"
    )
    if tfds_trn_path.exists():
        ds_trn = tf.data.experimental.load(
            str(tfds_trn_path),
            (
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
            ),
        )
    else:
        df_trn = pd.read_json(
            data_path / "codesearchnet_java" / "train.jsonl",
            orient="records",
            lines=True,
        )[:n]
        ds_trn = convert_df_to_tfds(df_trn, tokenizer, max_length, batch_size)
        tfds_trn_path.mkdir(exist_ok=True)
        tf.data.experimental.save(ds_trn, str(tfds_trn_path))
        del df_trn

    tfds_val_path = (
        data_path / "codesearchnet_java" / f"tfds_val_{max_length}len_{batch_size}bs"
    )
    if tfds_val_path.exists():
        ds_val = tf.data.experimental.load(
            str(tfds_val_path),
            (
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
            ),
        )
    else:
        df_val = pd.read_json(
            data_path / "codesearchnet_java" / "valid.jsonl",
            orient="records",
            lines=True,
        )[:n]
        ds_val = convert_df_to_tfds(df_val, tokenizer, max_length, batch_size)
        tfds_val_path.mkdir(exist_ok=True)
        tf.data.experimental.save(ds_val, str(tfds_val_path))
        del df_val

    logging.info("Starting the training of all RNN based models.")
    # Train RNN based models
    for config in _RNN_CONFIGs:
        rnn_model = RNNModel(
            config["rnn_type"],
            config["n_layers"],
            tokenizer.get_vocab_size(),
            config["embedding_dim"],
            config["rnn_units"],
            batch_size,
            str(out_path),
            tokenizer,
        )
        rnn_model.train(ds_trn, ds_val, epochs)
        rnn_model.save()

    logging.info("Starting the training of all Transformer based models.")
    # Train Transformer models
    pass

In [ ]:
data_path = Path("/tmp/data/codesearchnet")
models_path = Path("/tmp/models")
rnn_path = models_path / "rnn_layers1_vocab10000_embed256_units1024"
EPOCHS = 2
MAX_LEN = 100
BS = 8
N = 100
_RNN_CONFIGs = [VANILLA_CONFIG]

train(
    data_path=data_path,
    out_path=models_path,
    epochs=EPOCHS,
    max_length=MAX_LEN,
    batch_size=BS,
    n=N,
)

assert (models_path / "tokenizer.json").exists()
assert rnn_path.exists()
assert (rnn_path / "assets").exists()
assert (rnn_path / "model_config.json").exists()
assert (rnn_path / "saved_model.pb").exists()
assert (rnn_path / "variables").exists()
assert EPOCHS == len(list(rnn_path.glob("*.index")))

INFO:root:Loading tokenizer from /tmp/models/tokenizer.json.
INFO:root:Starting the training of all RNN based models.


Epoch 1/2
12/12 [==============================] - 8s 656ms/step - loss: 7.2490 - val_loss: 5.1655
Epoch 2/2
12/12 [==============================] - 7s 625ms/step - loss: 5.2661 - val_loss: 5.2514
INFO:tensorflow:Assets written to: /tmp/models/rnn_layers1_vocab10000_embed256_units1024/assets


INFO:tensorflow:Assets written to: /tmp/models/rnn_layers1_vocab10000_embed256_units1024/assets
INFO:root:Starting the training of all Transformer based models.


In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.core.ipynb.
Converted 01_data.transforms.ipynb.
Converted 02_model.core.ipynb.
Converted 04_evaluation.core.ipynb.
Converted index.ipynb.
